In [1]:
import pandas as pd
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import matplotlib.pyplot as plt


In [2]:
data = pd.read_csv('IMDB Dataset.csv')

In [3]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
data.shape

(50000, 2)

In [5]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [6]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

#### Result before cleaning

In [7]:
data['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

# Text CLeaning

### we need to remove special characters
### We need to filter out the stop words
### we need to split the text into tokens

In [8]:
#using re.compile pattenstring is converted to patten objects
no_space = re.compile("[.;:!\'?,\"()\[\]]")
space = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

##array of stopwords
stop_words = (stopwords.words('english'))


#to tokenize using regular expression method
tokenizer=RegexpTokenizer(r'\w+')

stemmer = SnowballStemmer(language='english')

#words are reduced to its root form using stemming method


def data_cleaning(review):
    clean_review = review.lower() #converted each letter to lower case
    clean_review = no_space.sub("",review) # all special character beloning to no_space objects replaced by ""
    clean_review = space.sub("",review) # all special character beloning to space objects replaced by " "
    tokens = tokenizer.tokenize(clean_review)
    clean_tokens =[]
    
    #stemming is done to reduce word to its root form
    for token in tokens:
        if token not in stop_words:
            clean_tokens.append(token)
    
    stem_tokens=[]
    
    for token in clean_tokens:
        stem_tokens.append(stemmer.stem(token))
        
    clean_review=' '.join(stem_tokens)

        
    return clean_review

#### Result after cleaning

In [9]:
data_cleaning(data['review'][1])

'a wonder littl product the film techniqu unassum oldtimebbc fashion give comfort sometim discomfort sens realism entir piec the actor extrem well chosen michael sheen got polari voic pat you truli see seamless edit guid refer william diari entri well worth watch terrif written perform piec a master product one great master comedi life the realism realli come home littl thing fantasi guard rather use tradit dream techniqu remain solid disappear it play knowledg sens particular scene concern orton halliwel set particular flat halliwel mural decor everi surfac terribl well done'

In [11]:
data['review'].apply(data_cleaning)

0        one review mention watch 1 oz episod hook they...
1        a wonder littl product the film techniqu unass...
2        i thought wonder way spend time hot summer wee...
3        basic famili littl boy jake think zombi closet...
4        petter mattei love time money visual stun film...
                               ...                        
49995    i thought movi right good job it creativ origi...
49996    bad plot bad dialogu bad act idiot direct anno...
49997    i cathol taught parochi elementari school nun ...
49998    i go disagre previous comment side maltin one ...
49999    no one expect star trek movi high art fan expe...
Name: review, Length: 50000, dtype: object

### Splitting train and test set

In [12]:
train_reviews=data.review[:40000]
train_sentiments=data.sentiment[:40000]

In [13]:
test_reviews=data.review[40000:]
test_sentiments=data.sentiment[40000:]

## Binary vectorization

###### Binary Term Frequency captures presence (1) or absence (0) of term in document

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True, ngram_range=(1, 3))
cv.fit(train_reviews)


In [ ]:
train_reviews = cv.transform(train_reviews)
test_reviews = cv.transform(test_reviews)

## Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(train_reviews,train_sentiments)

In [ ]:
Y_predict = log_reg.predict(test_reviews)
Y_proba = log_reg.predict_proba(test_reviews)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cf_matrix=confusion_matrix(test_sentiments, Y_predict)

### visualizing confusion matrix

In [ ]:
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='plasma')

#### precision- recall

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(test_sentiments, Y_predict,average="binary", pos_label="positive")

In [ ]:
recall_score(test_sentiments, Y_predict,average="binary", pos_label="positive")

In [ ]:
from sklearn.metrics import f1_score
f1_score(test_sentiments, Y_predict,average="binary", pos_label="positive")

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
cr = classification_report(test_sentiments,Y_predict,target_names=['Positive','Negative'])

In [ ]:
print(cr)

In [ ]:
train_reviews[0]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_sentiments,Y_predict)

In [ ]:
cv.get_feature_names

In [ ]:
feature_to_coef = {
    word : coef for word, coef in zip(cv.get_feature_names(), log_reg.coef_[0])
}

In [ ]:
feature_to_coef

In [ ]:
log_reg.coef_[0]

In [ ]:
for best_positive in sorted(
        feature_to_coef.items(),
        key=lambda x: x[1],
        reverse=True)[:5]:
    print(best_positive)